In [ ]:
import os
import pickle
import numpy as np
import detectron2
import torch
import mrcnn_utils
import mrcnn_visualize
import pathlib
import skimage
import skimage.io
import evaluate

import analyze
import matplotlib.pyplot as plt

In [ ]:
with open('../data/outputs_np.pickle', 'rb') as f:
    pred_data = pickle.load(f)
    
gt_json_path = pathlib.Path('../data/raw/via_2.0.8/via_powder_particle_masks.json')
gt_ddicts = analyze.get_data_dicts(gt_json_path)

keys = list(pred_data.keys())

In [ ]:
# ground truth
i = 3
gt = analyze.extract_gt(gt_ddicts[i])
img = gt['img']
boxes = gt['bboxes']
masks = gt['masks']
class_ids = gt['class_idx']
colors = evaluate.random_colors(len(class_ids), 32)
mrcnn_visualize.display_instances(img, boxes, masks, class_ids, [''], colors=colors)

In [ ]:
## predictions with watershed, binary closing, and merging- fill in original masks (closes larger gaps)
i = -1
flist = list(pred_data.keys())
instance = pred_data[flist[-1]][0]
pmasks = np.transpose(instance['masks'], (1,2,0))
pclass_ids = instance['class']
masks_ws_merge, im_no_bg, = evaluate.refine_masks_watershed_li(img, pmasks, True, True, (0,0,0), merge=True)
boxes_ws_merge = mrcnn_utils.extract_bboxes(masks_ws_merge)

mrcnn_visualize.display_instances(im_no_bg, boxes_ws_merge, masks_ws_merge, pclass_ids, [''], colors=colors)

In [ ]:
results = analyze.fast_instance_match(masks, pmasks)

In [ ]:
i = 28
m_idx = results['gt_tp'][i]
fig, ax = plt.subplots(1,2)
ax[0].imshow(masks[:,:,m_idx[0]])
ax[1].imshow(pmasks[:,:,m_idx[1]])

In [ ]:
plt.plot(results['IOU_match'])

In [ ]:
pmasks_new = analyze.filter_mask_size(pmasks)

In [ ]:
results.keys()

In [ ]:
results

In [ ]:
dir(evaluate)